In [ ]:
import os
import math
import time
import copy
import random
import numpy as np
import torch.nn as nn
import torch.autograd
from skimage import io
from torch import optim
import torch.nn.functional as F
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader

working_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
working_path = os.path.join(working_path, 'Scannet-original')

CHECKPOINTDIR = os.path.abspath('/storage/scratch3/buddhiw-change-detection/SCanNet_Checkpoints/SECOND')

BEST_ORIGINAL_MODEL = os.path.join(CHECKPOINTDIR, 'best_original_model.pth')

from utils.loss import CrossEntropyLoss2d, weighted_BCE_logits, ChangeSimilarity
from utils.utils import accuracy, SCDD_eval_all, AverageMeter
from datasets import RS_ST as RS
#from models.TED import TED as Net
from models.SCanNet import SCanNet as Net
NET_NAME = 'SCanNet_Original'
DATA_NAME = 'ST'

In [ ]:
net = Net(3, num_classes=RS.num_classes).cuda()
net.load_state_dict(torch.load(args['load_path']), strict=False)
net.eval()

In [ ]:
DATA_DIR = os.path.dirname(os.path.dirname(working_path))
root = os.path.join(DATA_DIR, 'Datasets', 'SECOND')
print('Loading dataset from %s' % root)

In [ ]:
image_1 = io.imread(os.path.join(root, 'test', 'T1', '04548.png'))
image_2 = io.imread(os.path.join(root, 'test', 'T2', '04548.png'))

label_1 = io.imread(os.path.join(root, 'test', 'GT_T1', '04548.png'))
label_2 = io.imread(os.path.join(root, 'test', 'GT_T2', '04548.png'))

image_1, image_2, label_1, label_2 = F.to_tensor(image_1), F.to_tensor(image_2), F.to_tensor(label_1), F.to_tensor(label_2)


In [ ]:
imgs_A, imgs_B, labels_A, labels_B, imgs_id = data
if args['gpu']:
    imgs_A = imgs_A.cuda().float()
    imgs_B = imgs_B.cuda().float()
    labels_A = labels_A.cuda().long()
    labels_B = labels_B.cuda().long()

labels_A = labels_A.cpu().detach().numpy()
labels_B = labels_B.cpu().detach().numpy()
outputs_A = outputs_A.cpu().detach()
outputs_B = outputs_B.cpu().detach()
change_mask = F.sigmoid(out_change).cpu().detach() > 0.5
preds_A = torch.argmax(outputs_A, dim=1)
preds_B = torch.argmax(outputs_B, dim=1)
preds_A = (preds_A * change_mask.squeeze().long()).numpy()
preds_B = (preds_B * change_mask.squeeze().long()).numpy()

pred_A_color = RS.Index2Color(preds_A[0])
pred_B_color = RS.Index2Color(preds_B[0])
io.imsave(os.path.join(args['pred_dir'], NET_NAME + '_A.png'), pred_A_color)
io.imsave(os.path.join(args['pred_dir'], NET_NAME + '_B.png'), pred_B_color)
print('Prediction saved!')